In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import StratifiedKFold
import xgboost
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb
from sklearn import ensemble
from sklearn.metrics import roc_auc_score
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('Week8_train.csv')
df_test = pd.read_csv('Week8_test.csv') 

In [219]:
df.drop(['PARTY_ID'], axis =1, inplace =True)
df_test.drop(['PARTY_ID'], axis =1, inplace =True)

In [220]:
df.select_dtypes(object).columns

Index(['V_245', 'V_271', 'V_301', 'V_304', 'V_316', 'V_400', 'V_410', 'V_434'], dtype='object')

In [221]:
df_test.select_dtypes(object).columns

Index(['V_245', 'V_271', 'V_301', 'V_304', 'V_316', 'V_400', 'V_410', 'V_434'], dtype='object')

In [222]:
def ohe(df, col, variables):
    for i in variables:
        df[col+'_'+i] = np.where(df[col]==i, 1,0)

In [223]:
df.V_301.fillna('Financials', inplace=True)
df_test.V_301.fillna('Financials', inplace=True)
df.V_434.fillna('Commercial Banks', inplace=True)
df_test.V_434.fillna('Commercial Banks', inplace=True)
df.dropna(axis = 0, how= 'any', inplace = True)

In [224]:
cols = ['V_245', 'V_271', 'V_301', 'V_304', 'V_316', 'V_400', 'V_410', 'V_434']

In [225]:
l1 = df.V_245.unique()
l2= df.V_271.unique()
l3= df.V_301.unique()
l4= df.V_304.unique()
l5= df.V_316.unique()
l6= df.V_400.unique()
l7= df.V_410.unique()
l8= df.V_434.unique()

In [226]:
ohe(df, 'V_245', l1 )
ohe(df, 'V_271', l2 )
ohe(df, 'V_301', l3 )
ohe(df, 'V_304', l4 )
ohe(df, 'V_316', l5 )
ohe(df, 'V_400', l6 )
ohe(df, 'V_410', l7 )
ohe(df, 'V_434', l8 )

In [227]:
ohe(df_test, 'V_245', l1 )
ohe(df_test, 'V_271', l2 )
ohe(df_test, 'V_301', l3 )
ohe(df_test, 'V_304', l4 )
ohe(df_test, 'V_316', l5 )
ohe(df_test, 'V_400', l6 )
ohe(df_test, 'V_410', l7 )
ohe(df_test, 'V_434', l8 )

In [228]:
df.drop(['V_245', 'V_271', 'V_301', 'V_304', 'V_316', 'V_400', 'V_410', 'V_434'], axis=1, inplace= True)
df_test.drop(['V_245', 'V_271', 'V_301', 'V_304', 'V_316', 'V_400', 'V_410', 'V_434'], axis=1, inplace= True)

In [229]:
X= df.drop(['id', 'TARGET'], axis=1)
y = df.TARGET

In [56]:
def create_folds(dfv, num_splits):
    dfv["kfold"] = -1
    dfv = dfv.sample(frac=1).reset_index(drop=True)
    kf = StratifiedKFold(n_splits=num_splits)
    for f, (t_, v_) in enumerate(kf.split(X = X, y= y)):
        dfv.loc[v_, 'kfold'] = f
    return dfv

In [57]:
df = create_folds(df, 3)

In [59]:
rf_test_pred = []
rf_valid_pred = {}
gbm_test_pred = []
gbm_valid_pred = {}
xgb_test_pred = []
xgb_valid_pred = {}
lg_test_pred = []
lg_valid_pred = {}

In [60]:
df.kfold.value_counts()

0    36446
1    36446
2    36446
Name: kfold, dtype: int64

In [126]:
for i in range(3):
    dftrain = df[df.kfold!=i].reset_index(drop = True)
    dfvalid = df[df.kfold==i].reset_index(drop = True)
    xtest = df_test.copy()
    
    xtrain = dftrain.drop(['TARGET', 'kfold'], axis= 1)
    xvalid = dfvalid.drop (['TARGET', 'kfold'], axis= 1)
    ytrain = dftrain.TARGET
    yvalid = dfvalid.TARGET
    
    valid_ids = xvalid['id'].values.tolist()
    
    undersample = RandomUnderSampler(sampling_strategy= 0.5 , random_state = 5)
    xtrain,ytrain = undersample.fit_resample(xtrain,ytrain)
    
    model_1 = ensemble.RandomForestClassifier()
        
    model_1.fit(xtrain, ytrain)
    preds_valid_1 = model_1.predict_proba(xvalid)[:,1]
    test_preds_1 = model_1.predict_proba(xtest)[:,1]
    rf_test_pred.append(test_preds_1)
    rf_valid_pred.update(dict(zip(valid_ids, preds_valid_1)))
    
    model_2 = ensemble.GradientBoostingClassifier()
        
    model_2.fit(xtrain,ytrain)
    preds_valid_2 = model_2.predict_proba(xvalid)[:,1]
    test_preds_2 = model_2.predict_proba(xtest)[:,1]
    gbm_test_pred.append(test_preds_2)
    gbm_valid_pred.update(dict(zip(valid_ids, preds_valid_2)))
    
    model_3 = xgboost.XGBClassifier()
        
    model_3.fit(xtrain,ytrain)
    preds_valid_3 = model_3.predict_proba(xvalid)[:,1]
    test_preds_3 = model_3.predict_proba(xtest)[:,1]
    xgb_test_pred.append(test_preds_3)
    xgb_valid_pred.update(dict(zip(valid_ids, preds_valid_3)))
    

In [127]:
from sklearn.preprocessing import MinMaxScaler
scale = MinMaxScaler()
df_scale = df.copy()
df_scale.iloc[:,1:-1] = scale.fit_transform(df_scale.iloc[:, 1:-1])

In [128]:
dft_scale = df_test.copy()
dft_scale.iloc[:,1:] = scale.fit_transform(dft_scale.iloc[:, 1:])

In [129]:
for i in range(3):
    dftrain = df[df.kfold!=i].reset_index(drop = True)
    dfvalid = df[df.kfold==i].reset_index(drop = True)
    xtest = df_test.copy()
    
    xtrain = dftrain.drop(['TARGET', 'kfold'], axis= 1)
    xvalid = dfvalid.drop (['TARGET', 'kfold'], axis= 1)
    ytrain = dftrain.TARGET
    yvalid = dfvalid.TARGET
    
    valid_ids = xvalid['id'].values.tolist()
    
    undersample = RandomUnderSampler(sampling_strategy= 0.5 , random_state = 5)
    xtrain,ytrain = undersample.fit_resample(xtrain,ytrain)
    
    model_4 = LogisticRegression()
        
    model_4.fit(xtrain,ytrain)
    preds_valid_4 = model_4.predict_proba(xvalid)[:,1]
    test_preds_4 = model_4.predict_proba(xtest)[:,1]
    lg_test_pred.append(test_preds_4)
    lg_valid_pred.update(dict(zip(valid_ids, preds_valid_4)))

In [130]:
df.columns.get_loc('TARGET')

473

In [168]:
sample= pd.DataFrame(df_test.iloc[:,0])

In [149]:
df = df.merge(rf_valid_pred, on = 'id', how = 'left')

In [145]:
gbm_valid_pred = pd.DataFrame.from_dict(gbm_valid_pred, orient="index").reset_index()
gbm_valid_pred.columns = ["id", "pred_2"]
df = df.merge(gbm_valid_pred, on = 'id', how = 'left')

xgb_valid_pred = pd.DataFrame.from_dict(xgb_valid_pred, orient="index").reset_index()
xgb_valid_pred.columns = ["id", "pred_3"]
df = df.merge(xgb_valid_pred, on = 'id', how = 'left')

lg_valid_pred = pd.DataFrame.from_dict(lg_valid_pred, orient="index").reset_index()
lg_valid_pred.columns = ["id", "pred_4"]
df = df.merge(lg_valid_pred, on = 'id', how = 'left')

In [150]:
XTRAIN_PRED = df['pred_1','pred_2','pred_3', 'pred_4']
Y_TARGET = df.TARGET

,id,V_1,V_2,V_3,V_4,V_5,V_6,V_7,V_8,V_9,...,V_434_Health Care Technology,V_434_Real Estate Investment Trusts,V_434_Provincial,V_434_Internet & Catalog Retail,V_434_Libraries,kfold,pred_2,pred_3,pred_4,pred_1
0,64724,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0.212555,0.159930,0.356627,0.27
1,92844,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0.264353,0.252283,0.340507,0.28
2,83681,0,0,0,0,1,0,0,2,0,...,0,0,0,0,0,0,0.197989,0.175165,0.326517,0.18
3,72086,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.332618,0.380384,0.739301,0.43
4,78464,0,0,0,0,14,0,0,0,0,...,0,0,0,0,0,0,0.342136,0.297252,0.329870,0.35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109333,105819,0,0,0,0,3,0,0,0,0,...,0,0,0,0,0,2,0.185373,0.157750,0.344623,0.13
109334,21707,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,0.185373,0.165851,0.383343,0.15
109335,88990,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,0.260545,0.285394,0.334555,0.29
109336,76791,0,0,0,0,10,0,0,0,0,...,0,0,0,0,0,2,0.498814,0.435545,0.338023,0.35


In [184]:
sample['pred_1'] = np.mean(np.column_stack(rf_test_pred), axis=1)
sample['pred_2'] = np.mean(np.column_stack(gbm_test_pred), axis=1)
sample['pred_3'] = np.mean(np.column_stack(xgb_test_pred), axis=1)
sample['pred_4'] = np.mean(np.column_stack(lg_test_pred), axis=1)

In [185]:
df_test = df_test.merge(sample, on = 'id', how = 'left')

In [189]:
useful_features = ["pred_1", "pred_2",'pred_3','pred_4']
df_test = df_test[useful_features]

final_predictions = []
scores = []
for fold in range(3):
    xtrain =  df[df['kfold'] != fold].reset_index(drop=True)
    xvalid = df[df['kfold'] == fold].reset_index(drop=True)
    xtest = df_test.copy()


    ytrain = xtrain['TARGET']

    yvalid = xvalid['TARGET']
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    
    model = LogisticRegression()
    model.fit(xtrain, ytrain)
    
    preds_valid = model.predict_proba(xvalid)[:,1]
    test_preds = model.predict_proba(xtest)[:,1]
    final_predictions.append(test_preds)
    print(fold, roc_auc_score(yvalid, preds_valid))


0 0.7071140001262979
1 0.704338665300735
2 0.7067148352447231


In [190]:
preds = np.mean(np.column_stack(final_predictions), axis=1)

In [208]:
testfile = pd.read_csv('Week8_test.csv', usecols = ['id'])
testfile

,id
0,0
1,1
2,2
3,3
4,4
...,...
110894,110894
110895,110895
110896,110896
110897,110897


In [243]:
testfile['TARGET'] = preds

In [244]:
testfile['TARGET'] = np.where(testfile['TARGET'] <0.5 , 0,1)

In [215]:
testfile.to_csv("submission1.csv", index=False)

# ATTEMPT 2

In [234]:
X= df.drop(['id', 'TARGET'], axis=1)
y = df.TARGET

In [238]:
undersample = RandomUnderSampler(sampling_strategy= 0.5 , random_state = 5)
xtrain1,ytrain1 = undersample.fit_resample(X,y)

In [239]:
xgb = xgboost.XGBClassifier()
X_train, X_test, y_train, y_test = train_test_split(xtrain1, ytrain1, test_size= 0.25, random_state= 42)

In [245]:
xgb.fit(X_train, y_train)
pred1 =train_pred = xgb.predict(X_test)    

In [247]:
print(roc_auc_score(y_test,pred1))

0.620786056506263


In [249]:
test_set = df_test.drop(['id'], axis =1)

In [251]:
pred1f = xgb.predict(test_set)

In [261]:
testfile1= pd.DataFrame(df_test['id'])
testfile1['TARGET'] = pred1f

In [262]:
testfile1.to_csv("submission2.csv", index=False)

# ATTEMPT 2.5

In [275]:
from sklearn.ensemble import ExtraTreesClassifier
extratree = ExtraTreesClassifier()
extratree.fit(X_train, y_train)
pred2_ =train_pred = extratree.predict(X_test)    
print(roc_auc_score(y_test,pred2_))
test_set = df_test.drop(['id'], axis =1)
pred2f = extratree.predict(test_set)

0.5991526872873707


In [276]:
testfile2_= pd.DataFrame(df_test['id'])
testfile2_['TARGET'] = pred2f
testfile2_.to_csv("submission2__.csv", index=False)

# ATTEMPT 3

In [3]:
df3 = pd.read_csv('Week8_train.csv')
df_test3 = pd.read_csv('Week8_test.csv') 

df3.drop(['PARTY_ID'], axis =1, inplace =True)
df_test3.drop(['PARTY_ID'], axis =1, inplace =True)


In [4]:
df3.select_dtypes(object).columns

Index(['V_245', 'V_271', 'V_301', 'V_304', 'V_316', 'V_400', 'V_410', 'V_434'], dtype='object')

In [5]:
df_test3.select_dtypes(object).columns

Index(['V_245', 'V_271', 'V_301', 'V_304', 'V_316', 'V_400', 'V_410', 'V_434'], dtype='object')

In [6]:
nan = [i for i in df3.columns if df3[i].isna().sum().any()]
nan

['V_301', 'V_304', 'V_434']

In [7]:
#categorical
df3.V_304.isna().sum(), df3.V_304.unique()

(1561, array(['F', 'M', nan], dtype=object))

In [8]:
#categorical
df3.V_301.isna().sum(), df3.V_301.nunique()

(6331, 15)

In [9]:
df_test3.V_301.isna().sum(), df3.V_301.nunique()

(6205, 15)

In [10]:
#categorical
df3.V_434.isnull().sum(), df3.V_434.nunique()

(6335, 85)

In [11]:
#categorical
df_test3.V_434.isnull().sum(), df_test3.V_434.nunique()

(6335, 84)

In [12]:
df3.V_301 = df3.V_301.fillna(value = 'Undefined')
df3.V_434 = df3.V_434.fillna(value = 'Undefined')

df_test3.V_301 = df3.V_301.fillna(value = 'Undefined')
df_test3.V_434 = df3.V_434.fillna(value = 'Undefined')

In [13]:
df3.V_304 = df3.V_304.fillna(value = 'M')

df_test3.V_304 = df3.V_304.fillna(value = 'M')

In [14]:
cols

NameError: name 'cols' is not defined

In [ ]:
L1= pd.get_dummies(df3['V_245'], prefix='V_245', drop_first= True)
L2= pd.get_dummies(df3['V_271'], prefix='V_271', drop_first= True)
L3= pd.get_dummies(df3['V_301'], prefix='V_301', drop_first= True)
L4= pd.get_dummies(df3['V_316'], prefix='V_316', drop_first= True)
L5= pd.get_dummies(df3['V_400'], prefix='V_400', drop_first= True)
L6= pd.get_dummies(df3['V_410'], prefix='V_410', drop_first= True)
L7= pd.get_dummies(df3['V_434'], prefix='V_434', drop_first= True)
L8= pd.get_dummies(df3['V_304'], prefix='V_304', drop_first= True)

In [ ]:
df3 = pd.concat([df3, L1,L2,L3,L4,L5,L6,L7,L8], axis =1)
df3

In [ ]:
Lt1= pd.get_dummies(df_test3['V_245'], prefix='V_245', drop_first= True)
Lt2= pd.get_dummies(df_test3['V_271'], prefix='V_271', drop_first= True)
Lt3= pd.get_dummies(df_test3['V_301'], prefix='V_301', drop_first= True)
Lt4= pd.get_dummies(df_test3['V_316'], prefix='V_316', drop_first= True)
Lt5= pd.get_dummies(df_test3['V_400'], prefix='V_400', drop_first= True)
Lt6= pd.get_dummies(df_test3['V_410'], prefix='V_410', drop_first= True)
Lt7= pd.get_dummies(df_test3['V_434'], prefix='V_434', drop_first= True)
Lt8= pd.get_dummies(df_test3['V_304'], prefix='V_304', drop_first= True)

df_test3 = pd.concat([df_test3, Lt1,Lt2,Lt3,Lt4,Lt5,Lt6,Lt7,Lt8], axis =1)
df_test3

In [ ]:
df3.drop(['V_245', 'V_271', 'V_301', 'V_304', 'V_316', 'V_400', 'V_410', 'V_434'], axis=1, inplace= True)
df_test3.drop(['V_245', 'V_271', 'V_301', 'V_304', 'V_316', 'V_400', 'V_410', 'V_434'], axis=1, inplace= True)

In [20]:
from sklearn.model_selection import GridSearchCV, KFold

In [ ]:
X= df3.drop(['id','TARGET'], axis =1)
y=df3.TARGET
from xgboost import XGBClassifier
undersample = RandomUnderSampler(sampling_strategy= 0.5 , random_state = 5)
xtrain3,ytrain3 = undersample.fit_resample(X,y)

X_train, X_test, y_train, y_test = train_test_split(xtrain3, ytrain3, test_size= 0.25, random_state= 42)

params = {
    'n_estimators': [40, 50, 100],
    'learning_rate': [0.01,0.05,0.075],
    'booster': ['gbtree', 'gblinear'],
    'gamma': [0, 0.5, 1],
}

gs2 = GridSearchCV(XGBClassifier(n_jobs=-1), params, n_jobs=-1, cv=KFold(n_splits=3), scoring='roc_auc')
gs2.fit(X_train, y_train)

print('Best score:', gs2.best_score_)
print('Best score:', gs2.best_params_)

In [314]:
xgb.fit(X_train, y_train)
pred3 =train_pred = xgb.predict(X_test)    

print(roc_auc_score(y_test,pred3))

test_set = df_test3.drop(['id'], axis =1)

pred3f = xgb.predict(test_set)

testfile3= pd.DataFrame(df_test3['id'])
testfile3['TARGET'] = pred3f

testfile3.to_csv("submission3.csv", index=False)

(110899, 595)

(110899, 594)